In [1]:
import pandas as pd
import numpy as np
import os
from neo4j import GraphDatabase
import getpass
from dotenv import load_dotenv



In [2]:
df = pd.read_excel('PitchBook_Company_General_Information_2024_06_28_12_35_25.xlsx', header=7)
df = df[~df['Company Name'].isna()]

column_names = list(df.columns)
print(column_names)
df.head()

['Company ID', 'Company Name', 'Company Former Name', 'Description', 'Financing Status Note', 'Primary Industry Sector', 'Primary Industry Group', 'Primary Industry Code', 'Verticals', 'All Industries', 'Website', '# of Employees at Company', 'Former Listing', 'Company Business Status', 'Company Financing Status', 'Year Founded', 'Parent Company', 'HQ Address Line 1', 'HQ Address Line 2', 'HQ City', 'HQ State/Province', 'HQ Post Code', 'HQ Country/Territory', 'HQ Phone', 'HQ Fax', 'HQ Email', 'Primary Contact', 'Primary Contact Title', 'Secondary Contact', 'PitchBook Link', 'LinkedIn URL']


,Company ID,Company Name,Company Former Name,Description,Financing Status Note,Primary Industry Sector,Primary Industry Group,Primary Industry Code,Verticals,All Industries,...,HQ Post Code,HQ Country/Territory,HQ Phone,HQ Fax,HQ Email,Primary Contact,Primary Contact Title,Secondary Contact,PitchBook Link,LinkedIn URL
0,55185-04,Estimize,NaN,Developer of an open financial estimates platf...,The company was acquired by ExtractAlpha for a...,Information Technology,Software,Financial Software,"FinTech, SaaS, TMT",Financial Software*;Other Financial Services;M...,...,10001,United States,+1 (212) 634-9085,NaN,info@estimize.com,Vinesh Jha,Chief Executive Officer,NaN,NaN,NaN
1,167096-35,JetClosing,NaN,Developer of a real estate settlement applicat...,The company is no longer actively in business....,Consumer Products and Services (B2C),Services (Non-Financial),Real Estate Services (B2C),"Mobile, Real Estate Technology, SaaS, TMT",Real Estate Services (B2C)*;Other Commercial S...,...,98154,United States,+1 (866) 538-1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,56288-62,New Constructs,NaN,Operator of an investment research firm intend...,"Solidus, Lucius Burch, Lhoist North America of...",Information Technology,Software,Financial Software,"Artificial Intelligence & Machine Learning, Bi...",Financial Software*;Media and Information Serv...,...,37027,United States,+1 (615) 377-0443,NaN,info@newconstructs.com,David Trainer,Co-Founder & Chief Executive Officer,NaN,NaN,NaN
3,59990-50,Justworks,Clockwork,Developer of a human resource management platf...,The company raised $16.84 million of venture f...,Information Technology,Software,Business/Productivity Software,"FinTech, HR Tech, SaaS",Business/Productivity Software*;Human Capital ...,...,10008-7119,United States,+1 (888) 534-1711,NaN,hello@justworks.com,Isaac Oates,Co-Founder & Executive Chairman,NaN,NaN,NaN
4,53739-01,Procore Technologies,NaN,Procore Technologies Inc is a cloud-based cons...,The company raised $634.49 million in its init...,Information Technology,Software,Business/Productivity Software,"Construction Technology, Real Estate Technolog...",Business/Productivity Software*;Media and Info...,...,93013,United States,+1 (866) 477-6267,NaN,info@procore.com,Paul Lyandres,President,Craig Courtemanche Jr.,NaN,NaN


In [3]:
# Sanitize property names
def process_column_name(name):
    name = name.lower()
    return name.replace(' ', '_').replace('#', 'number').replace('/', '_').replace(';', '_').replace('-', '_').replace(',', '')
    

In [7]:
processed_column_names = [process_column_name(i) for i in column_names]
column_mapper = dict(zip(column_names, processed_column_names))
df = df.rename(column_mapper, axis=1)

df_fintech = df[df['verticals'].str.contains('FinTech')]
list(df_fintech.website)


['www.estimize.com',
 'www.newconstructs.com',
 'www.justworks.com',
 'www.driveway.ai',
 'www.magnite.com',
 'www.gusto.com',
 'www.carta.com',
 'www.yapstone.com',
 'www.cardlytics.com',
 'www.pitchbook.com',
 'www.mindbodyonline.com',
 'www.plaid.com',
 'www.humaninterest.com',
 'www.agro.club',
 'www.chargebee.com',
 'www.workmarket.com',
 'www.xactlycorp.com',
 'www.catalisgov.com',
 'www.coinflip.network',
 'www.floqast.com',
 'www.jibehealth.com',
 'www.zenefits.com',
 'www.pinnaclera.com',
 'www.quantexa.com',
 'www.ooyala.com',
 'www.tradeshift.com',
 'www.landbay.co.uk',
 'www.acumatica.com',
 'www.ocrolus.com',
 'www.softheon.com',
 'www.side.com',
 'www.karbonhq.com',
 'www.gosite.com',
 'www.showingtime.com',
 'www.tangocard.com',
 'www.beacon.com',
 'www.axial.net',
 'www.pollinate.global',
 'www.limelighthealth.com',
 'www.peopleconnect.us',
 'www.shyftmoving.com',
 'www.rezolve.com',
 'www.clarafinds.com',
 'www.oildex.com',
 'www.pinpoint.ai',
 'www.ivendi.com',
 'www.

In [5]:
df_fintech

,company_id,company_name,company_former_name,description,financing_status_note,primary_industry_sector,primary_industry_group,primary_industry_code,verticals,all_industries,...,hq_post_code,hq_country_territory,hq_phone,hq_fax,hq_email,primary_contact,primary_contact_title,secondary_contact,pitchbook_link,linkedin_url
0,55185-04,Estimize,NaN,Developer of an open financial estimates platf...,The company was acquired by ExtractAlpha for a...,Information Technology,Software,Financial Software,"FinTech, SaaS, TMT",Financial Software*;Other Financial Services;M...,...,10001,United States,+1 (212) 634-9085,NaN,info@estimize.com,Vinesh Jha,Chief Executive Officer,NaN,NaN,NaN
2,56288-62,New Constructs,NaN,Operator of an investment research firm intend...,"Solidus, Lucius Burch, Lhoist North America of...",Information Technology,Software,Financial Software,"Artificial Intelligence & Machine Learning, Bi...",Financial Software*;Media and Information Serv...,...,37027,United States,+1 (615) 377-0443,NaN,info@newconstructs.com,David Trainer,Co-Founder & Chief Executive Officer,NaN,NaN,NaN
3,59990-50,Justworks,Clockwork,Developer of a human resource management platf...,The company raised $16.84 million of venture f...,Information Technology,Software,Business/Productivity Software,"FinTech, HR Tech, SaaS",Business/Productivity Software*;Human Capital ...,...,10008-7119,United States,+1 (888) 534-1711,NaN,hello@justworks.com,Isaac Oates,Co-Founder & Executive Chairman,NaN,NaN,NaN
5,58716-37,Driveway (Business/Productivity Software),NaN,Developer of a telematics technology designed ...,The company was acquired by Earnix for an undi...,Information Technology,Software,Business/Productivity Software,"Artificial Intelligence & Machine Learning, Fi...",Business/Productivity Software*;Media and Info...,...,92656,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,54118-36,Magnite,"The Rubicon Project, Inc., Rubicon Project, Inc.",Magnite is one of the largest supply-side plat...,The company completed a $540 million of debt r...,Information Technology,Software,Business/Productivity Software,"AdTech, FinTech, SaaS, TMT",Business/Productivity Software*;Media and Info...,...,10001,United States,+1 (212) 243-2769,+1 (212) 414-8748,contact@magnite.com,David Day,Chief Financial Officer,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,433414-45,YearEnd,NaN,Developer of taxation software intended to opt...,The company was acquired by Carta (Financial S...,Business Products and Services (B2B),Commercial Services,"Accounting, Audit and Tax Services (B2B)","FinTech, SaaS","Accounting, Audit and Tax Services (B2B)*;Fina...",...,94111,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,471280-69,YesRef,NaN,Developer of a referee payment portal designed...,"The company raised GBP 50,000 of venture fundi...",Information Technology,Software,Financial Software,"FinTech, Mobile, SaaS, TMT",Financial Software*;Business/Productivity Soft...,...,W1W 5PF,United Kingdom,+44 (0)33 0124 6844,NaN,NaN,Oliver Ballinger,"Co-Founder, Chief Executive Officer & Board Me...",NaN,NaN,NaN
1053,459859-42,Yottled,NaN,Developer of a client management platform desi...,M25 sold a stake in the company to an undisclo...,Information Technology,Software,Business/Productivity Software,"FinTech, Mobile, SaaS",Business/Productivity Software*;Application So...,...,48111,United States,+1 (831) 228-6756,NaN,hello@yottled.com,NaN,NaN,NaN,NaN,NaN
1054,435817-09,YouAttest,NaN,Developer of a cloud-based IGA engine designed...,"The company raised $655,000 of venture funding...",Business Products and Services (B2B),Commercial Services,"Accounting, Audit and Tax Services (B2B)","Artificial Intelligence & Machine Learning, Bi...","Accounting, Audit and Tax Services (B2B)*;Netw...",...,92618,United States,+1 (714) 658-0765,NaN,info@youattest.com,Garret Grajek,"Co-Founder, Chief Executive Officer, President...",NaN,NaN,NaN


In [9]:
print(df['Description'].iloc[0])
print(df['Website'].iloc[0])
print(df['HQ State/Province'].iloc[0])

Developer of an open financial estimates platform designed to provide an accurate, more timely and more representative view of expectations. The company's platform collects forward-looking financial estimates from independent, buy-side, and sell-side analysts, along with those of private investors and academics as well as offers a view of market expectations on stocks and economic indicators, enabling investors to analyze the market before investing.
www.estimize.com
New York


In [39]:
import json
import pandas as pd
from neo4j import GraphDatabase

# Escape single quotes for Cypher queries
def escape_quotes_for_cypher(value):
    return value.replace("'", "\\'")

# Sanitize property names
def sanitize_property_name(name):
    return name.replace(' ', '_').replace('#', 'number_of').replace('/', '_').replace(';', '_').replace('-', '_').replace(',', '')

# Generate Cypher CREATE statements from DataFrame
def df_to_cypher_create_nodes(df, node_label_column, attribute_columns):
    """
    Generates Cypher CREATE statements for nodes from a DataFrame.

    Args:
    df (pd.DataFrame): DataFrame containing the data.
    node_label_column (str): Column name for node labels.
    attribute_columns (list): List of column names for node attributes.

    Returns:
    list: List of Cypher CREATE statements.
    """
    cypher_statements = []
    
    for _, row in df.iterrows():
        properties = []
        node_label = 'Company'
        properties.append(f"name: '{escape_quotes_for_cypher(row[node_label_column])}'")  # Ensure 'name' property is set

        for col in attribute_columns:
            sanitized_col = sanitize_property_name(col)
            value = row[col]
            if isinstance(value, (dict, list)):
                json_value = json.dumps(value)
                escaped_value = escape_quotes_for_cypher(json_value)
                value = f"'{escaped_value}'"
            elif isinstance(value, bool):
                value = str(value).lower()
            elif isinstance(value, str):
                escaped_value = escape_quotes_for_cypher(value)
                value = f"'{escaped_value}'"
            else:
                value = str(value)
            properties.append(f"{sanitized_col}: {value}")

        properties_str = ", ".join(properties)
        create_statement = f"CREATE (:{sanitize_property_name(node_label)} {{{properties_str}}})"
        cypher_statements.append(create_statement)
    
    return cypher_statements


In [40]:
cypher_statements = df_to_cypher_create_nodes(df, 'Company Name', ['Company Name', 'Company Former Name', 'Description',
       'Financing Status Note', 'Primary Industry Sector',
       'Primary Industry Group', 'Primary Industry Code', 'Verticals',
       'All Industries', 'Website', '# of Employees at Company'])

print(cypher_statements)

["CREATE (:Company {name: 'Gusto', Company_Name: 'Gusto', Company_Former_Name: 'Switchboard Labs, ZenPayroll', Description: 'Developer of cloud-based payroll, benefits, and human resource management platform designed to offer and manage employee benefits. The company\\'s platform offers features including payroll management, benefits management, team management, time tracking, health insurance, dental insurance, and vision insurance management, enabling small businesses to take care of their team\\'s requirements.', Financing_Status_Note: 'The company raised $230 million of Series E venture funding in a deal led by Dragoneer Investment Group and T. Rowe Price on May 12, 2022, putting the company\\'s pre-money valuation at $9.4 billion. Franklin Venture Partners and 26 other investors also participated in the round. The funds will be used to continue to expand operations and business reach.', Primary_Industry_Sector: 'Information Technology', Primary_Industry_Group: 'Software', Primary_

In [41]:
# Upload nodes to Neo4j
def upload_nodes_to_neo4j(tx, statements):
    for statement in statements:
        tx.run(statement)

from neo4j import GraphDatabase

# Establish a connection to the Neo4j database
uri = "bolt://localhost:7687"
username = "neo4j"
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(uri, auth=(username, password))

with driver.session() as session:
        session.write_transaction(upload_nodes_to_neo4j, cypher_statements)
driver.close()



/var/folders/l5/2302s0j90898y2c73n3g1wmr0000gn/T/ipykernel_27423/57142294.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(upload_nodes_to_neo4j, cypher_statements)


In [35]:
# Connect to a Neo4j instance
from langchain_community.graphs import Neo4jGraph
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password=os.getenv('NEO4J_PASSWORD') 
)

result = graph.query("""
MATCH (n) 
RETURN n
""")

print(result)

[{'n': {'number_of_of_Employees_at_Company': 1300.0, 'Primary_Industry_Sector': 'Business Products and Services (B2B)', 'Description': 'Developer of financial information software intended to serve the financial and professional services firms. The company offers specialist news such as merger and acquisition (M&A) and business intelligence information, research, analysis and data on financial markets, enabling clients in the advisory, corporate and financial communities to stay updated with all the financial happenings taking place globally.', 'name': 'Acuris', 'Financing_Status_Note': 'The company was acquired by ION Group, via its financial sponsor The Carlyle Group and its management, through a GBP 1.38 billion LBO in July 2019. As a part of the transaction, HPS Investment Partners and The Goldman Sachs Group provided GBP 1.11254 million of debt financing in the form of Term loan B and GBP 20 million of revolving credit facility to support the deal.', 'Company_Former_Name': 'Merger

In [42]:



llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_KEY'),
                 temperature = 0, 
                 model_name = "gpt-4o")
llm_transformer = LLMGraphTransformer(llm)

text = """
"Gusto's platform offers features including payroll management, benefits management, team management, time tracking, health insurance, dental insurance, and vision insurance management, enabling small businesses to take care of their team's requirements."
"""

# Create a template to help guide the LLM towards the type of graph we want to build
template = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            ## 1. Overview
            You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
            Given a parapraph, you need to extract the company and its services. The relationship between company and its service is OFFER
            """
        )
    ]
)

documents = [Document(page_content=text)]


llm_transformer_props = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Company", "Product", "Service"],
    allowed_relationships=["OFFER"],
    # node_properties=["born_year"],
    # prompt = template
)
graph_documents_props = llm_transformer_props.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents_props[0].nodes}")
print(f"Relationships:{graph_documents_props[0].relationships}")



Nodes:[Node(id='Gusto', type='Company'), Node(id='Payroll Management', type='Service'), Node(id='Benefits Management', type='Service'), Node(id='Team Management', type='Service'), Node(id='Time Tracking', type='Service'), Node(id='Health Insurance', type='Service'), Node(id='Dental Insurance', type='Service'), Node(id='Vision Insurance Management', type='Service')]
Relationships:[Relationship(source=Node(id='Gusto', type='Company'), target=Node(id='Payroll Management', type='Service'), type='OFFER'), Relationship(source=Node(id='Gusto', type='Company'), target=Node(id='Benefits Management', type='Service'), type='OFFER'), Relationship(source=Node(id='Gusto', type='Company'), target=Node(id='Team Management', type='Service'), type='OFFER'), Relationship(source=Node(id='Gusto', type='Company'), target=Node(id='Time Tracking', type='Service'), type='OFFER'), Relationship(source=Node(id='Gusto', type='Company'), target=Node(id='Health Insurance', type='Service'), type='OFFER'), Relationshi

In [43]:
graph.add_graph_documents(graph_documents_props)

In [ ]:
from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship
)
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )
    

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
    [(
      "system",
      f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination."""),
        ("human", "Use the given format to extract information from the following input: {input}"),
        ("human", "Tip: Make sure to answer in the correct format"),
    ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)


def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.run(document.page_content)
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])